In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets

import matplotlib.pyplot as plt

/Users/cjkuk/fall23/cs6780/project/jude-max/proj_venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)
# train is 6000 long
# test is 1000 long

In [11]:
# Initialize network and optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Function to train the model
def train(epoch, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        print()

In [12]:
# Function to test the model
def test(test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct / len(test_loader.dataset)

In [16]:
# Train and test for each pair of digits
accuracies = []
for i in range(9):
    for j in range(i+1, 10):
        train_loader = torch.utils.data.DataLoader([x for x in train_dataset if x[1] == i or x[1] == j])
        test_loader = torch.utils.data.DataLoader([x for x in test_dataset if x[1] == i or x[1] == j])
        for epoch in range(1):
            train(epoch, train_loader)
        accuracies.append(test(test_loader))

# Plot accuracies
plt.plot(accuracies)
plt.xlabel('Task number')
plt.ylabel('Accuracy')
plt.show()

Train Epoch: 0 [0/12665 (0%)]	Loss: nan
Train Epoch: 0 [0/11881 (0%)]	Loss: nan
Train Epoch: 0 [0/12054 (0%)]	Loss: nan


KeyboardInterrupt: 

In [ ]:
accuracies = []
for i in range(9):
    for j in range(i+1, 10):
        train_loader = torch.utils.data.DataLoader([x for x in train_dataset if x[1] == i or x[1] == j])
        test_loader = torch.utils.data.DataLoader([x for x in test_dataset if x[1] == i or x[1] == j])
        train(epoch, train_loader)
        accuracies.append(test(test_loader))